# Homework 1
***

We are going to work with the following dataset: fluid current in a tube.
Some statistics are collected for dataset, incl. mean, skewness, kurtosis, etc. We are predicting flow rate ('tohn/hour'). We need to build confidence and predictive intervals.

In [25]:
import numpy as np
from sklearn import datasets, linear_model, preprocessing, model_selection
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import t, sem

In [26]:
df = pd.read_csv('exxsol_data.csv', sep=';', header=(0))

There are 10 features and 1 label to predict:

In [27]:
df.head()

,mean,std,skew,kurt,RMS,crest,freq_peak,shan,perm,temp,tohn/hour
0,0.025639,0.946336,-0.034895,-1.491265,0.946674,0.888424,526.01052,9.584955,1.000464,21.14,3.49
1,0.025639,0.946336,-0.034895,-1.491265,0.946674,0.888424,526.01052,9.584955,1.000464,21.14,3.49
2,0.016856,0.952439,-0.037338,-1.504434,0.952579,0.943467,530.01060,9.402718,0.991088,21.14,3.49
3,0.016856,0.952439,-0.037338,-1.504434,0.952579,0.943467,530.01060,9.402718,0.991088,21.14,3.49
4,0.031998,0.972590,-0.056116,-1.546518,0.973106,0.921600,256.00512,9.020444,0.973701,21.14,3.56


In [28]:
y = df['tohn/hour']
freq_temp = df[['freq_peak','temp']]

Physics tells us that flow rate is a function of a frequency peak and temperature.

In [29]:
freq_temp, y = shuffle(freq_temp, y)

# split data into training and testing sets
#from sklearn.model_selection import train_test_split
#train_freq, test_freq, train_y, test_y = train_test_split(freq, y, train_size=0.7, random_state=2)

lr = linear_model.LinearRegression()
predicted = model_selection.cross_val_predict(lr, freq_temp, y.ravel(), cv=20)
score = model_selection.cross_val_score(lr, freq_temp, y, scoring='r2',cv=20)

## Q0: Build point estimate for mean r2 score and its deviation

In [31]:
mean = np.mean(score)
std = np.std(score)

print('Mean r2 score', mean)
print('Deviation r2 score', std)

Mean r2 score 0.8248989362962857
Deviation r2 score 0.0770382586118165


## Q1: Predicted is an array with predictions of the label y. Assuming, that $\sigma = 0.1$, compute 95% confidence and predictive interval for mean squared error. 

In [32]:
sigma = 0.1
conf = 0.95
z = 1.96 # for 95% confidence
mse = (predicted - y) ** 2
n = len(mse)
mse_mean = np.mean(mse)

print('n = ', n)
print('Confidence interval with a known sigma', mse_mean - z * sigma / np.sqrt(n), mse_mean + z * sigma / np.sqrt(n))
print('Predictive interval with a known sigma', mse_mean - conf * sigma, mse_mean + conf * sigma)

n =  700
Confidence interval with a known sigma 0.15582143538813895 0.17063764273010065
Predictive interval with a known sigma 0.0682295390591198 0.25822953905911983


## Q2:  Compute 95% confidence and predicted intervals for mean squared error, assuming no knowledge about sigma

In [33]:
mse_std = sem(mse)
t_a = t.ppf((1 + conf) / 2, n - 1)

print('Confidence interval with a unknowned sigma', mse_mean - t_a * mse_std / np.sqrt(n), mse_mean + t_a * mse_std / np.sqrt(n))
print('Predictive interval with a unknowned sigma', mse_mean - conf * mse_std, mse_mean + conf * mse_std)

Confidence interval with a unknowned sigma 0.16224904728311476 0.16421003083512484
Predictive interval with a unknowned sigma 0.15067745465669458 0.17578162346154502


We can use additional features and more complex model, e.g. ElasticNet.

In [34]:
y = df['tohn/hour']
X = df.drop(['tohn/hour'], axis=1)
X = preprocessing.scale(X)
X, y = shuffle(X, y)

encv = linear_model.ElasticNetCV(cv=10, max_iter=3000, n_alphas=10)
predicted_encv = model_selection.cross_val_predict(encv, X, y.ravel(), cv=20)
score_encv = model_selection.cross_val_score(encv, X, y.ravel(), scoring='r2', cv=20)

## Q3:  Compute 95% confidence interval for difference in means of mean squared error between 2 models, assuming no knowledge about $\sigma$.

In [35]:
conf = 0.95
diffs = (predicted - y) ** 2 - (predicted_encv - y) ** 2
n = len(diffs)
std_diffs = sem(diffs)
mean_diffs = np.mean(diffs)
t_a = t.ppf((1 + conf) / 2, n - 1)

print('Confidence interval', mean_diffs - t_a * std_diffs / np.sqrt(n), mean_diffs + t_a * std_diffs / np.sqrt(n))

Confidence interval 1.7573102888988776 1.7708648077689455
